In [6]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 125.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
###

%%writefile app.py
import streamlit as st
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

# Load the saved model
model = torch.load("/content/drive/MyDrive/Shortcuts/model_scripted.pt")
model.eval()  # Set the model in evaluation mode

def predict_class(image_path, model, class_labels):
    # Define image transformations to preprocess the input image
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.Grayscale(num_output_channels=3),  # Convert to RGB (3 channels)
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Load and preprocess the input image
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the input tensor to the same device as the model
    if torch.cuda.is_available():
        image = image.to("cuda")

    # Perform the prediction
    with torch.no_grad():
        output = model(image)

    # Get the predicted class index
    predicted_probabilities = F.softmax(output, dim=1).squeeze().tolist()
    predicted_classes = [class_labels[i] for i in range(len(class_labels))]

    return predicted_classes, predicted_probabilities

class_labels = {0: "glioma_tumor", 1: "meningioma_tumor", 2: "no_tumor", 3: "pituitary_tumor"}

# Streamlit UI
st.title("Brain Tumor Classification")
st.write("Upload an image for tumor classification.")
uploaded_image = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])

if uploaded_image is not None:
    # Display the uploaded image
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

    # Make a prediction when the user clicks the button
    if st.button("Predict"):
        with st.spinner("Predicting..."):
            # Save the uploaded image temporarily
            image_path = "temp_image.jpg"
            with open(image_path, "wb") as f:
                f.write(uploaded_image.read())

            # Predict the class
            predicted_classes, predicted_probabilities = predict_class(image_path, model, class_labels)

            # Find the class with the highest probability
            max_prob_index = predicted_probabilities.index(max(predicted_probabilities))
            max_prob_class = predicted_classes[max_prob_index]
            max_prob = predicted_probabilities[max_prob_index] * 100

            # Display the highest prediction using st.success
            st.success(f"Highest Prediction: {max_prob_class} ({max_prob:.2f}%)")

            # Display all class possibilities
            st.write("Class Probabilities:")
            for label, prob in zip(predicted_classes, predicted_probabilities):
                st.write(f"{label}: {prob * 100:.2f}%")

            # Display the probabilities in a column chart
            st.bar_chart({label: prob for label, prob in zip(predicted_classes, predicted_probabilities)})

            # Remove the temporary image
            # st.image(image_path, use_column_width=True)


Overwriting app.py


In [17]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [18]:
!wget -q -O - ipv4.icanhazip.com

104.154.206.43


In [24]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://104.154.206.43:8501

npx: installed 22 in 2.037s
your url is: https://long-sites-share.loca.lt
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:995: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:995: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"
  Stopping...
^C
